# Gibbs Sampling: esercitazione

Esempio tratto da [Ediwn Chen Blog](http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/)

In [1]:
import numpy as np

# First, create a document sets
docs = [
    ['apple', 'ios', 'mac', 'book'],
    ['apple', 'mac', 'book', 'apple', 'store'],
    ['banana', 'mango', 'fruit'],
    ['apple', 'fruit'],
    ['orange', 'strawberry'],
    ['apple', 'fruit', 'mac', 'ios']
]

Per LDA dobbiamo considerare ogni occorrenza delle parole all'interno di un topic, non il termine in sé ma in ciascuna istanza.

Per farlo possiamo dare un ID univoco a ogni documento e a ogni termine di ogni documento.
E.g. 'apple' nel secondo documento è 10

In [2]:
# decidiamo un numero fisso k di topics

K = 2
topics = dict([(i, [])for i in range(0,K)])


In [3]:
# Simplified collapsed Gibbs sampling

# Go through each document, and randomly assign each word in the document to one of the K topics.
docs_pos = []
for i, d in enumerate(docs):
    d_pos = []
    for w in d:
        t = np.random.choice(topics.keys())
        d_pos.append(t)
        topics[t].append(w)
    docs_pos.append(d_pos)

In [4]:
for k, words in topics.items():
    print k, words

0 ['mac', 'book', 'book', 'strawberry', 'apple', 'ios']
1 ['apple', 'ios', 'apple', 'mac', 'apple', 'store', 'banana', 'mango', 'fruit', 'apple', 'fruit', 'orange', 'fruit', 'mac']


In [5]:
for i, d in enumerate(docs):
    print d
    print docs_pos[i]

['apple', 'ios', 'mac', 'book']
[1, 1, 0, 0]
['apple', 'mac', 'book', 'apple', 'store']
[1, 1, 0, 1, 1]
['banana', 'mango', 'fruit']
[1, 1, 1]
['apple', 'fruit']
[1, 1]
['orange', 'strawberry']
[1, 0]
['apple', 'fruit', 'mac', 'ios']
[0, 1, 1, 0]


Notice that this random assignment already gives you both topic representations of all the documents and word distributions of all the topics (albeit not very good ones).

Ovvero adesso abbiamo ottenuto z, pertanto implicitamente abbiamo anche fi e theta. Assumo che alfa e beta siano tutti equiprobabili.

In [6]:
def theta(doc, topics, topic):
    doc_topics = {}
    for k, t_words in topics.items():
        doc_topics[k] = len([x for x in doc if x in t_words])
    return float(doc_topics[topic])/sum(doc_topics.values())

In [7]:
print theta(docs[1], topics, 1)

0.5


In [8]:
from collections import Counter

def phi(topics, word):
    word_topics = {}
    for k, t_words in topics.items():
        w_counter = dict(Counter(t_words).most_common())
        try:
            word_t = w_counter[word]
        except KeyError:
            word_t = 0
        word_topics[k] = word_t
    probs = np.zeros(len(word_topics))
    for i, k in enumerate(sorted(word_topics.keys())):
        probs[i] = float(word_topics[k]) / sum(word_topics.values())
    return probs

In [9]:
print phi(topics, 'fruit')

[ 0.  1.]


- So to improve on them, for each document d…
  - Go through each word w in d…
    - And for each topic t, compute two things: 1) p(topic t | document d) = the proportion of words in document d that are currently assigned to topic t, and 2) p(word w | topic t) = the proportion of assignments to topic t over all documents that come from this word w. Reassign w a new topic, where we choose topic t with probability p(topic t | document d) * p(word w | topic t) (according to our generative model, this is essentially the probability that topic t generated word w, so it makes sense that we resample the current word’s topic with this probability). (Also, I’m glossing over a couple of things here, in particular the use of priors/pseudocounts in these probabilities.)
    - In other words, in this step, we’re assuming that all topic assignments except for the current word in question are correct, and then updating the assignment of the current word using our model of how documents are generated.


Tre possibilità sulle probabilità:

- normalizzo il vettore `probs` a somma 1
- ottengo un valore di una distribuzione di Dirichlet `np.random.dirichlet(probs)`
- genero un rand entro il max di probs e prendo il valore corrispondente

```
print w, 'old to', old_assignment, 'new to', new_assignment
print w, p_norm
```

In [10]:
def get_random(data):
    data = sorted([(i, x) for i, x in enumerate(data)], key=lambda y: -y[1])
    r = np.random.uniform()
    s = 0
    for pos, value in data:
        s+= value
        if r <= s:
            break
    return pos
        
def pseudo_gibbs(docs, topics, pos):
    for i, doc in enumerate(docs):
        for j, w in enumerate(doc):
            probs = []
            for k in sorted(topics.keys()):
                p_topic_doc = theta(doc, topics, k)
                p_word_topic = phi(topics, w)[k]
                probs.append(p_topic_doc * p_word_topic)
            p_norm = np.array(probs)/sum(probs)
            old_assignment = pos[i][j]
            new_assignment = get_random(p_norm)
            topics[old_assignment].remove(w)
            topics[new_assignment].append(w)
            pos[i][j] = new_assignment

In [11]:
for i in range(0, 10):
    pseudo_gibbs(docs, topics, docs_pos)
    print phi(topics, 'apple')

[ 0.6  0.4]
[ 0.2  0.8]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]


In [12]:
for k, words in topics.items():
    print k, Counter(words).most_common()

0 [('book', 2), ('ios', 2), ('strawberry', 1)]
1 [('apple', 5), ('mac', 3), ('fruit', 3), ('mango', 1), ('orange', 1), ('banana', 1), ('store', 1)]
